In [3]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('trail_dataset.csv')

# Inspect columns for exact naming
print("Columns in dataset:", df.columns.tolist())

# Convert date columns to datetime with correct names
date_cols = ['screeningdate', 'enrollmentdate', 'randomizationdate', 'dropoutdate']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert boolean fields correctly (column names as per dataset)
bool_cols = ['hasdiabetes', 'hashypertension', 'hasheartdisease']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(bool)

# Drop duplicates
df_clean = df.drop_duplicates().copy()

# Filter out age outliers (valid age range 18-85)
if 'age' in df_clean.columns:
    df_clean = df_clean[(df_clean['age'] >= 18) & (df_clean['age'] <= 85)]

# Filter vital sign outliers (systolicbp 90-200, diastolicbp 60-120)
if 'misystolicbp' in df_clean.columns:
    df_clean = df_clean[(df_clean['misystolicbp'] >= 90) & (df_clean['misystolicbp'] <= 200)]
if 'midiastolicbp' in df_clean.columns:
    df_clean = df_clean[(df_clean['midiastolicbp'] >= 60) & (df_clean['midiastolicbp'] <= 120)]

# Filter glucose outliers (70-300 mg/dL)
if 'glucosemgdl' in df_clean.columns:
    df_clean = df_clean[(df_clean['glucosemgdl'] >= 70) & (df_clean['glucosemgdl'] <= 300)]

# Impute missing numerical values with median where numerical columns exist
num_cols = ['weightkg', 'heightcm', 'bmi', 'misystolicbp', 'midiastolicbp', 'hemoglobingdl', 'creatininemgdl', 'glucosemgdl']
for col in num_cols:
    if col in df_clean.columns:
        median_value = df_clean[col].median()
        df_clean[col].fillna(median_value, inplace=True)

# Impute missing categorical values with mode
cat_cols = ['gender', 'ethnicity', 'dropoutreason', 'country', 'siteid']
for col in cat_cols:
    if col in df_clean.columns:
        mode_val = df_clean[col].mode()
        if not mode_val.empty:
            df_clean[col].fillna(mode_val[0], inplace=True)

# Final check for missing values counts
print("Missing values after cleaning:")
print(df_clean.isnull().sum().sort_values(ascending=False))

# Final cleaned shape
print(f"Cleaned dataset shape: {df_clean.shape}")

# Save cleaned dataset for further use
df_clean.to_csv('trail_dataset2.csv', index=False)


Columns in dataset: ['patient_id', 'site_id', 'site_name', 'country', 'city', 'age', 'gender', 'ethnicity', 'screening_date', 'enrollment_date', 'randomization_date', 'dropout_date', 'dropout_reason', 'has_diabetes', 'has_hypertension', 'has_heart_disease', 'weight_kg', 'height_cm', 'bmi', 'systolic_bp', 'diastolic_bp', 'hemoglobin_gdl', 'creatinine_mgdl', 'glucose_mgdl', 'visit_completion_rate', 'missed_visits', 'medication_adherence', 'data_quality_score', 'age_group']
Missing values after cleaning:
dropout_date             1505
dropout_reason           1504
randomization_date        635
enrollment_date           436
creatinine_mgdl           275
hemoglobin_gdl            231
glucose_mgdl              213
systolic_bp               184
diastolic_bp              184
medication_adherence      181
visit_completion_rate     153
weight_kg                 120
height_cm                 106
city                       30
gender                      0
ethnicity                   0
screening_dat

C:\Users\abane\AppData\Local\Temp\ipykernel_16872\1974130703.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(median_value, inplace=True)
C:\Users\abane\AppData\Local\Temp\ipykernel_16872\1974130703.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [8]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('trail_dataset3.csv')

# Step 1: Convert dates to datetime dtype
date_cols = ['screening_date', 'enrollment_date', 'randomization_date', 'dropout_date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Step 2: Drop columns with >90% missing (dropout_date, dropout_reason)
to_drop = []
for col in ['dropout_date', 'dropout_reason']:
    if col in df.columns:
        missing_percent = df[col].isnull().mean() * 100
        if missing_percent > 90:
            to_drop.append(col)
df.drop(columns=to_drop, inplace=True)

# Step 3: Convert booleans to bool dtype
bool_cols = ['has_diabetes', 'has_hypertension', 'has_heart_disease']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(bool)

# Step 4: Remove duplicate records (if any)
df = df.drop_duplicates()

# Step 5: Filter age to valid range 18-85
if 'age' in df.columns:
    df = df[(df['age'] >= 18) & (df['age'] <= 85)]

# Step 6: Fix invalid numeric outliers for vitals and labs
# Blood pressure
if 'systolic_bp' in df.columns:
    df = df[(df['systolic_bp'] >= 90) & (df['systolic_bp'] <= 200)]
if 'diastolic_bp' in df.columns:
    df = df[(df['diastolic_bp'] >= 60) & (df['diastolic_bp'] <= 120)]

# Glucose
if 'glucose_mg_dl' in df.columns:
    df = df[(df['glucose_mg_dl'] >= 70) & (df['glucose_mg_dl'] <= 300)]

# Step 7: Impute missing numeric columns with median
numeric_cols = [
    'weight_kg', 'height_cm', 'bmi', 'systolic_bp', 'diastolic_bp',
    'hemoglobin_g_dl', 'creatinine_mg_dl', 'glucose_mg_dl',
    'medication_adherence', 'visit_completion_rate'
]

for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Step 8: Impute missing categorical columns with mode or 'Unknown'
categorical_cols = ['gender', 'ethnicity', 'country', 'site_id', 'city']
for col in categorical_cols:
    if col in df.columns:
        mode_val = df[col].mode()
        if not mode_val.empty:
            df[col] = df[col].fillna(mode_val[0])
        else:
            df[col] = df[col].fillna('Unknown')

# Step 9: Recalculate BMI from height and weight for consistency
if 'weight_kg' in df.columns and 'height_cm' in df.columns:
    df['height_m'] = df['height_cm'] / 100
    df['calculated_bmi'] = df['weight_kg'] / (df['height_m'] ** 2)

    # Replace BMI if difference > 3 units
    bmi_diff_mask = (df['bmi'] - df['calculated_bmi']).abs() > 3
    df.loc[bmi_diff_mask, 'bmi'] = df.loc[bmi_diff_mask, 'calculated_bmi']

    # Drop helper columns
    df.drop(columns=['height_m', 'calculated_bmi'], inplace=True)

# Step 10: Final check for missing data
missing_final = df.isnull().sum()
print("Missing values after cleaning:\n", missing_final)

# Step 11: Save cleaned dataset
df.to_csv('trail_dataset3_cleaned.csv', index=False)

print(f"Data cleaning complete. Cleaned dataset shape: {df.shape}")


Missing values after cleaning:
 patient_id                 0
site_id                    0
site_name                  0
country                    0
city                       0
age                        0
gender                     0
ethnicity                  0
screening_date             0
enrollment_date          332
randomization_date       500
has_diabetes               0
has_hypertension           0
has_heart_disease          0
weight_kg                  0
height_cm                  0
bmi                        0
systolic_bp                0
diastolic_bp               0
hemoglobin_gdl           172
creatinine_mgdl          209
glucose_mgdl             169
visit_completion_rate      0
missed_visits              0
medication_adherence       0
data_quality_score         0
age_group                  0
dtype: int64
Data cleaning complete. Cleaned dataset shape: (1180, 27)


In [11]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('trail_dataset_3.csv')

# Step 1: Convert date columns to datetime
date_columns = ['screening_date', 'enrollment_date', 'randomization_date']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Step 2: Convert categorical columns to category dtype
categorical_cols = ['gender', 'ethnicity', 'site_id', 'country', 'city']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

# Step 3: Remove duplicates
df = df.drop_duplicates()

# Step 4: Remove columns with more than 90% missing values
threshold = 0.9
cols_to_drop = [col for col in df.columns if df[col].isnull().mean() > threshold]
df.drop(columns=cols_to_drop, inplace=True)

# Step 5: Impute missing numeric columns with median
numeric_cols = ['weight_kg', 'height_cm', 'bmi', 'systolic_bp', 'diastolic_bp', 
                'hemoglobin_g_dl', 'creatinine_mg_dl', 'glucose_mg_dl', 
                'medication_adherence', 'visit_completion_rate']
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Step 6: Impute missing categorical columns with mode or 'Unknown'
for col in categorical_cols:
    if col in df.columns:
        mode_val = df[col].mode()
        fill_val = mode_val[0] if not mode_val.empty else 'Unknown'
        df[col] = df[col].fillna(fill_val)

# Step 7: Filter out invalid outliers in numeric data
if 'age' in df.columns:
    df = df[(df['age'] >= 18) & (df['age'] <= 85)]

if 'systolic_bp' in df.columns:
    df = df[(df['systolic_bp'] >= 90) & (df['systolic_bp'] <= 200)]

if 'diastolic_bp' in df.columns:
    df = df[(df['diastolic_bp'] >= 60) & (df['diastolic_bp'] <= 120)]

if 'glucose_mg_dl' in df.columns:
    df = df[(df['glucose_mg_dl'] >= 70) & (df['glucose_mg_dl'] <= 300)]

# Step 8: Recalculate BMI from weight and height, adjust large differences
if 'weight_kg' in df.columns and 'height_cm' in df.columns:
    df['height_m'] = df['height_cm'] / 100
    df['bmi_calc'] = df['weight_kg'] / (df['height_m'] ** 2)
    bmi_diff_mask = (df['bmi'] - df['bmi_calc']).abs() > 3
    df.loc[bmi_diff_mask, 'bmi'] = df.loc[bmi_diff_mask, 'bmi_calc']
    df.drop(columns=['height_m', 'bmi_calc'], inplace=True)

# Step 9: Final missing value check
print("Missing values after cleaning:")
print(df.isnull().sum())

# Step 10: Save cleaned dataset
df.to_csv('trail_dataset_3_cleaned.csv', index=False)
print(f"Data cleaning complete. Cleaned dataset shape: {df.shape}")


Missing values after cleaning:
patient_id                 0
site_id                    0
site_name                  0
country                    0
city                       0
age                        0
gender                     0
ethnicity                  0
screening_date             0
enrollment_date          332
randomization_date       500
has_diabetes               0
has_hypertension           0
has_heart_disease          0
weight_kg                  0
height_cm                  0
bmi                        0
systolic_bp                0
diastolic_bp               0
hemoglobin_gdl           172
creatinine_mgdl          209
glucose_mgdl             169
visit_completion_rate      0
missed_visits              0
medication_adherence       0
data_quality_score         0
age_group                  0
dtype: int64
Data cleaning complete. Cleaned dataset shape: (1180, 27)


In [13]:
import pandas as pd

# Load dataset
df = pd.read_csv('trail_dataset_41.csv')

# Convert date columns to datetime format
df['enrollment_date'] = pd.to_datetime(df['enrollment_date'], errors='coerce')
df['randomization_date'] = pd.to_datetime(df['randomization_date'], errors='coerce')

# Handle missing values in date columns
# Option 1: Keep nulls and visualize missingness
# Option 2: Fill missing dates with a placeholder or imputed value
# For example, filling with the earliest date in the dataset:
# df['enrollment_date'].fillna(df['enrollment_date'].min(), inplace=True)
# df['randomization_date'].fillna(df['randomization_date'].min(), inplace=True)

# Example: Drop rows with missing date values if you prefer
# df.dropna(subset=['enrollment_date', 'randomization_date'], inplace=True)

# Save the preprocessed dataset if needed
df.to_csv('preprocessed_trail_dataset.csv', index=False)

print("Data preprocessing complete. Ready for visualization.")


Data preprocessing complete. Ready for visualization.


In [12]:
import pandas as pd
import numpy as np

# ============================================
# COMPLETE DATA CLEANING CODE FOR TRAIL_DATASET_4.CSV
# ============================================

# Load dataset
df = pd.read_csv('trail_dataset_4.csv')

print("STARTING DATA CLEANING PROCESS...")
print(f"Original dataset shape: {df.shape}")

# Step 1: Convert date columns to datetime (correct column names)
print("\nStep 1: Converting date columns to datetime...")
date_cols = ['screening_date', 'enrollment_date', 'randomization_date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"Converted {col} to datetime")

# Step 2: Convert boolean columns to proper boolean type
print("\nStep 2: Converting boolean columns...")
bool_cols = ['has_diabetes', 'has_hypertension', 'has_heart_disease']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(bool)
        print(f"Converted {col} to boolean")

# Step 3: Remove duplicate rows
print("\nStep 3: Removing duplicates...")
initial_rows = len(df)
df = df.drop_duplicates()
duplicates_removed = initial_rows - len(df)
print(f"Removed {duplicates_removed} duplicate rows")

# Step 4: Handle missing values in numeric columns (use correct column names)
print("\nStep 4: Handling missing values in numeric columns...")
numeric_cols = ['weight_kg', 'height_cm', 'bmi', 'systolic_bp', 'diastolic_bp',
                'hemoglobin_gdl', 'creatinine_mgdl', 'glucose_mgdl',
                'medication_adherence', 'visit_completion_rate']

for col in numeric_cols:
    if col in df.columns:
        missing_before = df[col].isnull().sum()
        if missing_before > 0:
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            print(f"Imputed {missing_before} missing values in {col} with median: {median_val:.2f}")

# Step 5: Handle missing values in categorical columns
print("\nStep 5: Handling missing values in categorical columns...")
categorical_cols = ['gender', 'ethnicity', 'country', 'site_id', 'city']
for col in categorical_cols:
    if col in df.columns:
        missing_before = df[col].isnull().sum()
        if missing_before > 0:
            mode_val = df[col].mode()
            fill_val = mode_val[0] if not mode_val.empty else 'Unknown'
            df[col] = df[col].fillna(fill_val)
            print(f"Imputed {missing_before} missing values in {col} with mode: {fill_val}")

# Step 6: Filter out invalid outliers
print("\nStep 6: Removing invalid outliers...")
initial_rows = len(df)

# Age filter (18-85 years)
if 'age' in df.columns:
    df = df[(df['age'] >= 18) & (df['age'] <= 85)]

# Blood pressure filters
if 'systolic_bp' in df.columns:
    df = df[(df['systolic_bp'] >= 90) & (df['systolic_bp'] <= 200)]

if 'diastolic_bp' in df.columns:
    df = df[(df['diastolic_bp'] >= 60) & (df['diastolic_bp'] <= 120)]

# Glucose filter (remove negative values and extreme highs)
if 'glucose_mgdl' in df.columns:
    df = df[(df['glucose_mgdl'] >= 70) & (df['glucose_mgdl'] <= 300)]

outliers_removed = initial_rows - len(df)
print(f"Removed {outliers_removed} rows with invalid outliers")

# Step 7: Recalculate BMI and fix inconsistencies
print("\nStep 7: Recalculating BMI for consistency...")
if 'weight_kg' in df.columns and 'height_cm' in df.columns and 'bmi' in df.columns:
    df['height_m'] = df['height_cm'] / 100
    df['calculated_bmi'] = df['weight_kg'] / (df['height_m'] ** 2)
    
    bmi_diff_mask = (df['bmi'] - df['calculated_bmi']).abs() > 3
    inconsistent_bmis = bmi_diff_mask.sum()
    
    df.loc[bmi_diff_mask, 'bmi'] = df.loc[bmi_diff_mask, 'calculated_bmi']
    df.drop(columns=['height_m', 'calculated_bmi'], inplace=True)
    print(f"Fixed {inconsistent_bmis} inconsistent BMI values")

# Step 8: Final data validation
print("\nStep 8: Final data validation...")
missing_final = df.isnull().sum()
total_missing = missing_final.sum()
print(f"Total missing values after cleaning: {total_missing}")

# Step 9: Save cleaned dataset
output_filename = 'trail_dataset_4_cleaned.csv'
df.to_csv(output_filename, index=False)

print("\n" + "="*50)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("="*50)
print(f"Cleaned dataset shape: {df.shape}")
print(f"Cleaned dataset saved as: {output_filename}")


STARTING DATA CLEANING PROCESS...
Original dataset shape: (1180, 27)

Step 1: Converting date columns to datetime...
Converted screening_date to datetime
Converted enrollment_date to datetime
Converted randomization_date to datetime

Step 2: Converting boolean columns...
Converted has_diabetes to boolean
Converted has_hypertension to boolean
Converted has_heart_disease to boolean

Step 3: Removing duplicates...
Removed 0 duplicate rows

Step 4: Handling missing values in numeric columns...
Imputed 172 missing values in hemoglobin_gdl with median: 13.50
Imputed 209 missing values in creatinine_mgdl with median: 1.13
Imputed 169 missing values in glucose_mgdl with median: 109.30

Step 5: Handling missing values in categorical columns...

Step 6: Removing invalid outliers...
Removed 3 rows with invalid outliers

Step 7: Recalculating BMI for consistency...
Fixed 0 inconsistent BMI values

Step 8: Final data validation...
Total missing values after cleaning: 828

DATA CLEANING COMPLETED SU